In [1]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import mlflow
from sklearn.metrics import mean_squared_error
import pandas as pd

### Interacting with the MLflow tracking server
The MlflowClient object allows us to interact with...

- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions.
- To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [3]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [6]:
experiment_name = "nyc-experiment-2"
experiment_id = client.create_experiment(experiment_name)
print(f"Created Experiment ID: {experiment_id}")

Created Experiment ID: 2


In [7]:
existing_experiment = client.get_experiment_by_name(experiment_name)
if existing_experiment:
    experiment_id = existing_experiment.experiment_id
else:
    experiment_id = client.create_experiment(experiment_name)

Let's check the latest versions for the experiment with id 1...


In [9]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=['metrics.rmse ASC']
)

In [10]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 268cade8ecf94c5fb545b89be0b77331, rmse: 5.2205
run id: d01ddde800f041988f7ee9e60baad470, rmse: 5.2231
run id: 14d84639141a4b2aa7ec0f53cfefecd0, rmse: 5.2298
run id: a95bdbaab65145888cd0a10ba7002285, rmse: 5.2374
run id: 2edd066541ca4724af9669e60872c7d5, rmse: 5.2398


### Interacting with the Model Registry
In this section We will use the MlflowClient instance to:

1. Register a new version for the experiment nyc-taxi-regressor
2. Retrieve the latests versions of the model nyc-taxi-regressor and check that a new version 4 was created.
3. Transition the version 4 to "Staging" and adding annotations to it.

In [12]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [14]:
run_id = "268cade8ecf94c5fb545b89be0b77331"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="NYC-Taxi-Regressor")

Registered model 'NYC-Taxi-Regressor' already exists. Creating a new version of this model...
Created version '2' of model 'NYC-Taxi-Regressor'.


<ModelVersion: aliases=[], creation_timestamp=1742153779173, current_stage='None', description=None, last_updated_timestamp=1742153779173, name='NYC-Taxi-Regressor', run_id='268cade8ecf94c5fb545b89be0b77331', run_link=None, source='/home/muhammad/zoomcamp-mlops/02-experiment_tracking/mlruns/1/268cade8ecf94c5fb545b89be0b77331/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

Register a Model & Assign an Alias

In [21]:
model_name = "xgboost_model"

# Register the model name
try:
    client.create_registered_model(model_name)
    print(f"Model '{model_name}' registered successfully.")
except Exception as e:
    print(f"Model '{model_name}' might already exist. Error: {e}")

Model 'xgboost_model' registered successfully.


In [22]:
# Define model URI (ensure the model is saved)
model_uri = f"runs:/{run.info.run_id}/model"

# Create a new model version
model_version = client.create_model_version(name=model_name, source=model_uri, run_id=run.info.run_id)

print(f"Model '{model_name}' registered as version {model_version.version}")

Model 'xgboost_model' registered as version 1


In [23]:
client.set_registered_model_alias(model_name, "staging", model_version.version)
print(f"Model registered as version {model_version.version} with alias 'production'")

Model registered as version 1 with alias 'production'
